In [5]:
library(monocle3)
library(liger)
library(ggplot2)
library(purrr)
library(dplyr)

remove.not.expressed.features <- function(cds)
{
    return(cds[rowSums(counts(cds)) > 0,])
}




In [6]:
linnarson <- remove.not.expressed.features(readRDS('./objects/la-manno-embryo-object-raw.rds'))
metzakopian <- remove.not.expressed.features((readRDS('./objects/metza-model-object-raw.rds')))
welch <- remove.not.expressed.features(readRDS('./objects/welch-SN-object-raw.rds'))
saunder.homology.neurons <- remove.not.expressed.features(readRDS('./objects/saunders-homology-object-raw.rds'))
colData(linnarson)$Cell.Type = colData(linnarson)$Cell_type
colData(linnarson)$dataset = 'linnarson-embryo'

colData(saunder.homology.neurons)$dataset = 'saunder-neurons'
colData(saunder.homology.neurons)$Cell.Type = with(colData(saunder.homology.neurons), paste0(region,'_',cluster))

colData(metzakopian)$dataset = 'metzakopian'
colData(metzakopian)$Cell.Type = colData(metzakopian)$Cell.Types
colData(metzakopian)$region = 'in-vitro'
colData(metzakopian)$Timepoint = 'Day47'



In [7]:
analyze.monocle3 <- function(cds, object.name, correct, fields, genes, filter.genes = TRUE)
{
    print(dim(cds))
    if(filter.genes){
        starting.gene.set = rownames(cds)
        correct.values = unique(colData(cds)[[correct]])
        for (val in correct.values)
        {
            mask = colData(cds)[[correct]] == val
            # Everything that is expressed in 1% of the population
            cutoff = sum(mask) * 0.01
            cds.sub = cds[,mask]
            genes.mask = rowSums(counts(cds.sub)) > cutoff
            starting.gene.set = intersect(rownames(cds)[genes.mask], starting.gene.set)
        }
        print(paste('Analyzing',length(starting.gene.set), 'out of', length(rownames(cds))))
        cds <- preprocess_cds(cds, num_dim = 100, use_genes = starting.gene.set)
    }
    else
    {
        cds <- preprocess_cds(cds, num_dim = 100)
    }
    
    colData(cds)[[correct]] = as.factor(colData(cds)[[correct]])
    cds <- align_cds(cds, alignment_group = correct)
    
    
    q = plot_pc_variance_explained(cds)
    ggsave(plot = q, width = 4, height = 4, dpi = 300, filename = paste0('./plots/', object.name, '-monocle3-variance.png'))
    print('Reducing dimensionality')
    cds <- reduce_dimension(cds)
    print('Clustering cells')
    cds <- cluster_cells(cds)
    cds <- learn_graph(cds)
    for(f in fields)
    {
        cell.groups.show = T
        if(f == 'dataset'){
            cell.groups.show = F
        }
        q = plot_cells(cds, color_cells_by = f, label_cell_groups = cell.groups.show, show_trajectory_graph = F)
        ggsave(plot = q, width = 6, height = 6, dpi = 300, filename = paste0('./plots/', object.name, '-monocle3-',f,'.png'))
    }
    for(g in genes)
    {
        q = plot_cells(cds, genes = 'TH',show_trajectory_graph = F)
        ggsave(plot = q, width = 6, height = 6, dpi = 300,  filename =  paste0('./plots/genes', object.name, '-monocle3-',g,'.png'))
    }
    
    saveRDS(cds, paste0('./output/monocle3-',object.name,'.rds'))
    cds

}


merge_to_cds <- function(cds.list, name, fields=c('Cell.Type','dataset'),genes = c('TH'),correct = 'dataset'){
    library(purrr)
    library(dplyr)
    genes.list = purrr::reduce(cds.list, function(x, y){intersect(rownames(x),rownames(y))})
    
    l.cellmeta = map(cds.list, function(cds){as.data.frame(colData(cds))})
    cell_meta = bind_rows(l.cellmeta)
    rownames(cell_meta) =  purrr::reduce(l.cellmeta, function(x,y){c(rownames(x),rownames(y))})
    cds = new_cell_data_set(
        purrr::reduce(map(cds.list, function(cds){counts(cds)[genes.list,]}), cbind),
        cell_metadata = cell_meta
    )
    rownames(cds) = genes.list
    rowData(cds)$gene_short_name = genes.list
    analyze.monocle3(cds, name, 'dataset', fields, genes)
}


In [16]:
merge_to_cds(list(metzakopian, linnarson),'metza-linna',fields = c('dataset','Cell.Type','Timepoint'))
merge_to_cds(list(metzakopian, saunder.homology.neurons),'metza-saunders',fields = c('dataset','Cell.Type','region'))

merge_to_cds(list(welch, saunder.homology.neurons),'welch-saunders', fields = c('dataset','Cell.Type','region'),genes = c('TH','CALB1'))
merge_to_cds(list(welch, webber.neurons),'welch-webber-neurons',fields = c('dataset','Cell.Type','region'))





Warning message in new_cell_data_set(purrr::reduce(map(cds.list, function(cds) {:
"Warning: gene_metadata must contain a column verbatim named 'gene_short_name' for certain functions."


[1] 15582  3265
[1] "Analyzing 13123 out of 15582"


Aligning cells from different batches using Batchelor. 
Please remember to cite:
	 Haghverdi L, Lun ATL, Morgan MD, Marioni JC (2018). 'Batch effects in single-cell RNA-sequencing data are corrected by matching mutual nearest neighbors.' Nat. Biotechnol., 36(5), 421-427. doi: 10.1038/nbt.4091



[1] "Reducing dimensionality"


No preprocess_method specified, and aligned coordinates have been computed previously. Using preprocess_method = 'Aligned'



...

Warning message in min(data_df$weight[data_df$weight > 0]):
"no non-missing arguments to min; returning Inf"


class: cell_data_set 
dim: 11791 179730 
metadata(2): cds_version citations
assays(1): counts
rownames(11791): SAMD11 NOC2L ... MT-ND5 MT-ND6
rowData names(1): gene_short_name
colnames(179730): AAACCTGAGAGCTATA-1-0 AAACCTGAGATGGGTC-1-0 ...
  P60THRep6P2_AGGTGAATTCAT P60THRep6P2_GTGTGACTGGTT
colData names(28): genotype sample ... cluster subcluster
reducedDimNames(3): PCA Aligned UMAP
spikeNames(0):

Warning message in new_cell_data_set(purrr::reduce(map(cds.list, function(cds) {:
"Warning: gene_metadata must contain a column verbatim named 'gene_short_name' for certain functions."


[1]  14122 178899
[1] "Analyzing 9261 out of 14122"


ERROR: Error: cannot allocate vector of size 2.3 Gb
